# Getting Started with Exploring Segmentations 

## Before you start!

- This [notebook](getting-started-with-exploring-segmentations.ipynb) assumes that shapeworks conda environment has been activated using `conda activate shapeworks` on the terminal.
- See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb) to learn how to set up your environment to start using shapeworks library. Please note, the prerequisite steps will use the same code to setup the environment for this notebook and import `shapeworks` library.
- See [Getting Started with Segmentations](getting-started-with-segmentations.ipynb) to learn how to load and visualize binary segmentations.


## In this notebook, you will learn:

1. How to define your dataset location and explore what is available in it
2. How to explore your dataset
3. How to decide the grooming pipeline needed for your dataset

We will also define modular/generic helper functions as we walk through these items to reuse functionalities without duplicating code.

## Prerequisites

- Setting up `shapeworks` environment. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb). To avoid code clutter, the `setup_shapeworks_env` function can found in `Examples/Python/setupenv.py` module.
- Importing `shapeworks` library. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb).
- Helper functions for segmentations. See [Getting Started with Segmentations](getting-started-with-segmentations.ipynb).
- Helper functions for meshes. See [Getting Started with Meshes](getting-started-with-meshes.ipynb).
- Helper functions for visualization. See [Getting Started with Segmentations](getting-started-with-segmentations.ipynb), [Getting Started with Meshes](getting-started-with-meshes.ipynb), and [Getting Started with Exploring Segmentations](getting-started-with-exploring-segmentations.ipynb).



## Note about `shapeworks` APIs

shapeworks functions are inplace, i.e., `<swObject>.<function>()` applies that function to the `swObject` data. To keep the original data unchanged, you have first to copy it to another variable before applying the function.

## Notebook keyboard shortcuts

- `Esc + H`: displays a complete list of keyboard shortcuts
- `Esc + A`: insert new cell above the current cell
- `Esc + B`: insert new cell below the current cell
- `Esc + D + D`: delete current cell
- `Esc + Z`: undo
- `Shift + enter`: run current cell and move to next
- To show a function's argument list (i.e., signature), use `(` then `shift-tab`
- Use `shift-tab-tab` to show more help for a function
- To show the help of a function, use `help(function)` or `function?`
- To show all functions supported by an object, use `dot-tab` after the variable name



## Prerequisites

### Setting up `shapeworks` environment 

Here, we will append both your `PYTHONPATH` and your system `PATH` to setup shapeworks environment for this notebook. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb) for more details.

In this notebook, we assume the following.

- This notebook is located in `Examples/Python/notebooks/tutorials`
- You have built shapeworks from source in `build` directory within the shapeworks code directory

**Note:** If you run from a ShapeWorks installation, you don't need to set the `shapeworks_bin_dir`

In [ ]:
# import relevant libraries 
import sys 

# add parent-parent directory (where setupenv.py is) to python path
sys.path.insert(0,'../..')

# importing setupenv from Examples/Python
import setupenv

# indicate the bin directories for shapeworks and its dependencies
shapeworks_bin_dir = None # default
#shapeworks_bin_dir = "../../../../build/bin"

# set up shapeworks environment
setupenv.setup_shapeworks_env(shapeworks_bin_dir,  
                              verbose = False)

### Importing `shapeworks` library

In [ ]:
# let's import shapeworks library to test whether shapeworks is now set
try:
    import shapeworks as sw
except ImportError:
    print('ERROR: shapeworks library failed to import')
else:
    print('SUCCESS: shapeworks library is successfully imported!!!')

### Helper functions for segmentations

In [ ]:
# importing relevant libraries
import pyvista as pv
import numpy as np

# a helper function that converts shapeworks Image object to vtk image
def sw2vtkImage(swImg, verbose = False):
            
    # get the numpy array of the shapeworks image
    array  = swImg.toArray()
    
    # the numpy array needs to be permuted to match the shapeworks image dimensions
    array = np.transpose(array,(2,1,0))
    
    # converting a numpy array to a vtk image using pyvista's wrap function
    vtkImg = pv.wrap(array)
    
    if verbose:
        print('shapeworks image header information: ')
        print(swImg)

        print('\nvtk image header information: ')
        print(vtkImg) 
    
    return vtkImg

### Helper functions for meshes

In [ ]:
# importing relevant libraries
import os

# a helper function that converts shapeworks Mesh object to vtk mesh 
# TODO: to be modifed when #825 is addressed
def sw2vtkMesh(swMesh, verbose = False):
    
    if verbose:
        print('Header information: ')
        print(swMesh)

    # save mesh
    swMesh.write('temp.vtk')

    # read mesh into an itk mesh data
    vtkMesh = pv.read('temp.vtk')
    
    # remove the temp mesh file
    os.remove('temp.vtk')
    
    return vtkMesh

### Helper functions for visualization

In [ ]:
# importing itkwidgets to visualize single segmentations
import itkwidgets as itkw

# itkwidgets.view returns a Viewer object. And, the IPython Jupyter kernel 
# displays the last return value of a cell by default. So we have to use the display function
# to be able to call itkwidgets within a function and if statements
from IPython.display import display

# enable use_ipyvtk by default for interactive plots
pv.rcParams['use_ipyvtk'] = True 
    
# a helper function that addes a vtk image to a pyvista plotter
def add_volume_to_plotter( pvPlotter,      # pyvista plotter
                           vtkImg,         # vtk image to be added
                           rowIdx, colIdx, # subplot row and column index
                           title = None,   # text to be added to the subplot, use None to not show text 
                           shade_volumes  = True,  # use shading when performing volume rendering
                           color_map      = "coolwarm", # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
                           show_axes      = True,  # show a vtk axes widget for each rendering window
                           show_bounds    = False, # show volume bounding box
                           show_all_edges = True,  # add an unlabeled and unticked box at the boundaries of plot. 
                           font_size      = 10     # text font size for windows
                         ):
    
    # which subplot to add the volume to
    pvPlotter.subplot(rowIdx, colIdx)
    
    # add the volume
    pvPlotter.add_volume(vtkImg, 
                         shade   = shade_volumes, 
                         cmap    = color_map)

    if show_axes:
        pvPlotter.show_axes()

    if show_bounds:
        pvPlotter.show_bounds(all_edges = show_all_edges)

    # add a text to this subplot to indicate which volume is being visualized
    if title is not None:
        pvPlotter.add_text(title, font_size = font_size)
        
# a helper function that adds a mesh to a `pyvista` plotter.
def add_mesh_to_plotter( pvPlotter,      # pyvista plotter
                         vtkMesh,         # vtk mesh to be added
                         rowIdx, colIdx, # subplot row and column index
                         title = None,    # text to be added to the subplot, use None to not show text 
                         mesh_color      = "tan",  # string or 3 item list
                         mesh_style      = "surface", # visualization style of the mesh. style='surface', style='wireframe', style='points'. 
                         show_mesh_edges = False, # show mesh edges
                         opacity         = 1,
                         show_axes       = True,  # show a vtk axes widget for each rendering window
                         show_bounds     = False, # show volume bounding box
                         show_all_edges  = True,  # add an unlabeled and unticked box at the boundaries of plot. 
                         font_size       = 10     # text font size for windows
                         ):
    
    # which subplot to add the mesh to
    pvPlotter.subplot(rowIdx, colIdx)

    # add the surface mesh
    pvPlotter.add_mesh(vtkMesh, 
                       color      = mesh_color, 
                       style      = mesh_style,
                       show_edges = show_mesh_edges,
                       opacity    = opacity)

    if show_axes:
        pvPlotter.show_axes()

    if show_bounds:
        pvPlotter.show_bounds(all_edges = show_all_edges)

    # add a text to this subplot to indicate which volume is being visualized
    if title is not None:
        pvPlotter.add_text(title, font_size = font_size)

## 1. Defining and exploring your dataset

### Defining dataset location

You can download exemplar datasets from [ShapeWorks data portal](http://cibc1.sci.utah.edu:8080) after you login. For new users, you can [register](http://cibc1.sci.utah.edu:8080/#?dialog=register) an account for free. Please do not use an important password.

After you login, click `Collections` on the left panel and then `use-case-data-v2`. Select the dataset you would like to download by clicking on the checkbox on the left of the dataset name. See the video below.

**This notebook assumes that you have downloaded `ellipsoid-v2` in `Examples/Python/Data`.** Feel free to use your own dataset. 


<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/portal_data_download.mp4" autoplay muted loop controls style="width:100%"></p>

In [ ]:
# dataset name is the folder name for your dataset
datasetName  = 'ellipsoid-v2'

# path to the dataset where we can find shape data 
# here we assume shape data are given as binary segmentations
shapeDir      = '../../Data/' + datasetName + '/segmentations/'
    
print('Dataset Name:     ' + datasetName)
print('Shape Directory:  ' + shapeDir)

### What is available in the dataset?

First let's see how many shapes we have in the dataset.

**File formats:** For binary segmentations, all [itk-supported image formats](https://insightsoftwareconsortium.github.io/itk-js/docs/image_formats.html) can be used.

In [ ]:
import glob # for paths and file-directory search
from pathlib import Path # for generating robust paths irrespective of the platform:Win/Linux/Mac
# file extension for the shape data
shapeExtention = '.nrrd'

# let's get a list of files for available segmentations in this dataset
# * here is a wild character used to retrieve all filenames 
# in the shape directory with the file extensnion
shapeFilenames = sorted(glob.glob(shapeDir + '*' + shapeExtention)) 

print ('Number of shapes: ' + str(len(shapeFilenames)))
print('Shape files found:')
for shapeFilename in shapeFilenames:
    shapeFilename = Path(shapeFilename)
    print(shapeFilename)

## 2. Exploring your dataset

We would like to better understand the given dataset to decide the appropriate grooming (preprocessing) pipeline/step to prepare it for shape modeling.

### Loading your dataset

First step is to load the dataset. 

**Note:** If your dataset is large (large volumes and/or large number of segmentations), you could select a subset for this exploration step.

In [ ]:
# let's load the dataset - 

# list of shape segmentations
shapeSegList = []

# list of shape names (shape files prefixes) to be used 
# for saving outputs and visualizations
shapeNames   = [] 

# loop over all shape files and load individual segmentations
for shapeFilename in shapeFilenames:
    print('Loading: ' + shapeFilename)
    
    # current shape name
    segFilename = shapeFilename.split('/')[-1] 
    shapeName   = segFilename[:-len(shapeExtention)]
    shapeNames.append(shapeName)
    
    # load segmentation
    shapeSeg = sw.Image(shapeFilename)
    
    # append to the shape list
    shapeSegList.append(shapeSeg)

num_samples = len(shapeSegList)
print('\n' + str(num_samples) + 
      ' segmentations are loaded for the ' + datasetName + ' dataset ...')

### Visualizing your dataset

Now let's visualize all samples in a grid using `pyvista`. You may need to call `pv.close_all()` every once in a while to clean up the unclosed plotters.

First, we will define a helper function to determine the best grid size (rows and columns) given the number of samples in your dataset.

In [ ]:
# helper functions to define the best grid size for subplots
def postive_factors(num_samples):
    factors = []
    
    for whole_number in range(1, num_samples + 1):
        if num_samples % whole_number == 0:
            factors.append(whole_number)
    
    return factors

def num_subplots(num_samples):
    factors = postive_factors(num_samples)
    cols    = min(int(np.ceil(np.sqrt(num_samples))),max(factors))
    rows    = int(np.ceil(num_samples/cols))
    
    return rows, cols

Let's define a helper function that render all segmentations in your dataset. This function will initiate a `pyvista` plotter to render multiple windows, each with a single segmentation, add segmentations to the plotter, and start rendering.

In [ ]:
# helper function to add and plot a list of volumes
def plot_volumes(volumeList,           # list of shapeworks images to be visualized
                 volumeNames     = None,  # list of strings of same size as shape list used to add text for each plot window, use None to not show text per window 
                 use_same_window = False, # plot using multiple rendering windows if false
                 is_interactive  = True,  # to enable interactive plots
                 show_borders    = True,  # show borders for each rendering window
                 shade_volumes   = True,  # use shading when performing volume rendering
                 color_map       = "coolwarm", # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
                 show_axes       = True,  # show a vtk axes widget for each rendering window
                 show_bounds     = True,  # show volume bounding box
                 show_all_edges  = True,  # add an unlabeled and unticked box at the boundaries of plot. 
                 font_size       = 10,    # text font size for windows
                 link_views      = True   # link all rendering windows so that they share same camera and axes boundaries
                ):
    
    num_samples = len(volumeList)
    
    if volumeNames is not None:
        if use_same_window and (len(volumeNames) > 1):
            print('A single title needed when all volumes are to be displayed on the same window')
            return
        elif (not use_same_window) and (len(volumeNames) != num_samples):
            print('volumeNames list is not consistent with number of samples')
            return
            
    if use_same_window:
        grid_rows, grid_cols = 1, 1
    else:
        # define grid size for the given number of samples
        grid_rows, grid_cols  = num_subplots(num_samples)

    # define the plotter
    plotter = pv.Plotter(shape    = (grid_rows, grid_cols),
                         notebook = is_interactive, 
                         border   = show_borders) 
    
    # add the given volume list (one at a time) to the plotter
    for volumeIdx in range(num_samples):
        
        # which window to add the current volume
        if use_same_window:
            rowIdx, colIdx = 0, 0
            titleIdx       = 0
        else:
            idUnraveled     = np.unravel_index(volumeIdx, (grid_rows, grid_cols))
            rowIdx, colIdx  = idUnraveled[0], idUnraveled[1]
            titleIdx        = volumeIdx
        
        # which title to use
        if volumeNames is not None:
            volumeName = volumeNames[titleIdx]
        else:
            volumeName = None

        # convert sw image to vtk image
        if type(volumeList[volumeIdx]) == sw.Image:
            volume_vtk = sw2vtkImage(volumeList[volumeIdx], 
                                       verbose = False)
        else:
            volume_vtk = volumeList[volumeIdx]

        # add the current volume
        add_volume_to_plotter( plotter, volume_vtk,   
                               rowIdx = rowIdx, colIdx = colIdx, 
                               title          = volumeName,
                               shade_volumes  = shade_volumes, 
                               color_map      = color_map,
                               show_axes      = show_axes, 
                               show_bounds    = show_bounds, 
                               show_all_edges = show_all_edges, 
                               font_size      = font_size)
    # link views
    if link_views and (not use_same_window):
        plotter.link_views()  

    # now, time to render our volumes
    plotter.show(use_ipyvtk = is_interactive)

Now, let's define the parameters that controls the `pyvista` plotter and visualize the dataset.

In [ ]:
# define parameters that controls the plotter
use_same_window = False # plot using multiple rendering windows if false
is_interactive  = True  # to enable interactive plots
show_borders    = True  # show borders for each rendering window
shade_volumes   = True  # use shading when performing volume rendering
color_map       = "viridis" # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
show_axes       = True  # show a vtk axes widget for each rendering window
show_bounds     = True  # show volume bounding box
show_all_edges  = True  # add an unlabeled and unticked box at the boundaries of plot. 
font_size       = 10    # text font size for windows
link_views      = True  # link all rendering windows so that they share same camera and axes boundaries

# plot all segmentations in the shape list
plot_volumes(shapeSegList,    
             volumeNames     = shapeNames, 
             use_same_window = use_same_window,
             is_interactive  = is_interactive, 
             show_borders    = show_borders,  
             shade_volumes   = shade_volumes, 
             color_map       = color_map,
             show_axes       = show_axes,  
             show_bounds     = show_bounds,
             show_all_edges  = show_all_edges, 
             font_size       = font_size,   
             link_views      = link_views
             )

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-vols.mp4" autoplay muted loop controls style="width:100%"></p>

## 3. Deciding the grooming pipeline needed for your dataset

Does this dataset need grooming? What are grooming steps needed? Let's inspect the segmentations. What do we observe?

### Voxel spacing

Voxel spacing are not isotropic, i.e., voxel size in each of the three dimensions are not equal. This anisotropic spacing could adversally impact particles optimization since shapeworks assumes equal voxel spacing. Some datasets might also have different voxel spacings for each segmentation. 

*Hence, it is necessary to bring all segmentations to the same voxel spacing that is equal in all dimensions.* 

Another observation is voxel spacing is relatively large. This can be observed by the pixelated volume rendering and the jagged isosurface. 

*Hence, we can improve the segmentation resolution by decreasing voxel spacing.*

In [ ]:
# to better appreciate the pixelated nature of these segmentations, we need to only visualize 
# the binary segmentation, notice the thick slices

shapeIdx = 10
shapeSeg = shapeSegList[shapeIdx]
 
# to hide the rendered volume, drag the label map blend scroll
shapeSeg_vtk = sw2vtkImage(shapeSeg, verbose = True)

# to visualize label map - use label map blend 
itkw.view( image          = shapeSeg_vtk,  # for orthoginal image plane
           axes           = True,
           rotate         = True, # enable auto rotation
           interpolation  = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-voxel-space.mp4" autoplay muted loop controls style="width:100%"></p>

### Segmentations and image boundaries 

Some segmentations are very close to the image boundary, not leaving enough room for particles (correspondences) to move and spread over these surface regions. In particular, particles could overshoot outside the image boundary during optimization. 

Furthermore, if a segmentation touches the image boundary, this will result in an artificially (i.e., not real) open surface. 

*Hence, these segmentations needs to be padded with background voxels (zero-valued) to create more room along each dimension.*

In [ ]:
# let's inspect a segmentation that touches the image boundaries
shapeIdx = 3
shapeSeg = shapeSegList[shapeIdx]

shapeSeg_vtk = sw2vtkImage(shapeSeg, verbose = False)

# to visualize label map - use label map blend 
itkw.view( image          = shapeSeg_vtk, # for orthoginal image plane
           label_image    = shapeSeg_vtk,  # for volume rendering segmentation
           slicing_planes = True, 
           axes           = True,
           rotate         = True, # enable auto rotation
           interpolation  = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-seg-touch.mp4" autoplay muted loop controls style="width:100%"></p>


This segmentation touches the image boundary and hence will result in an artificially open surface. To inspect this behavior, we need extract a surface mesh (isosurface) from each segmentation. An isosurface is a three-dimensional surface that represents points of a constant value (aka isovalue) within the given volume of space.

In [ ]:
# let's see if there's a function that extracts an isosurface from an image
# use dot-tap to get a list of functions/apis available for shapeSeg

# found it - toMesh, let's see its help
help(shapeSeg.toMesh)

The `toMesh` function needs an isovalue, which is the constant value the represents the surface of interest. Since a shape segmentation is a binary image, the foreground is expected to have the value of 1 (white) and the background should have a zero value (black), so an appropriate isovalue to extract the foregound-background interface a value in between, e.g., 0.5

In [ ]:
# let's make sure that our assumptions about the voxel values are correct
# is the given volume a binary segmentation?

# first convert to numpy array
shapeSeg_array = shapeSeg.toArray()

# make sure that it is a binary segmentation
voxelValues = np.unique(shapeSeg_array)
print('\nVoxel values:' + str(voxelValues))

if len(voxelValues) > 2:
    print('WARNING: ' + shapeName + ' is not a bindary segmentation. Voxels have more than two distinct values')
    print('PLEASE make sure to use binary segmentations')
else:
    print('Shape ' + shapeName + ' is a binary segmentation')

In [ ]:
# now define the isovalue, in case a binary segmentation has a foreground label that is not 1
# we need to obtain a value inbetween

# get min and max values
minVal = shapeSeg_array.min()
maxVal = shapeSeg_array.max()

print('\nMinimum voxel value: ' + str(minVal))
print('Maximum voxel value: ' + str(maxVal))

isoValue = (maxVal - minVal)/2.0
print('\nisoValue = ' + str(isoValue))

In [ ]:
# let's extract the segmentation isosurface and visualize it

# extract isosurface
shapeMesh = shapeSeg.toMesh(isovalue = isoValue)
 
# sw to vtk
shapeMesh_vtk = sw2vtkMesh(shapeMesh)

# visualize with axes and auto rotation
itkw.view(  geometries     = shapeMesh_vtk, 
            rotate         = True, # enable auto rotation
            axes           = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-iso-touch.mp4" autoplay muted loop controls style="width:100%"></p>

So, we have been able to extract a segmentation's isosurface and visualize it as a surface mesh. It is worth noting that the jagged surface is due to the anisotropic voxel space (with spacing in z-dimension is double that of x- and y-dimensions) and large voxel size.

### Shape alignment

One can observe from the segmentation visualization that they are not roughly aligned, i.e., they do not share the same coordinate frame where each individual shape is located differently compared to other shapes. 

*Aligning shapes is a critical preprocessing step to avoid the shape model to encode variabilities pertaining to global transformations such as rotation and translation.* 

In [ ]:
# let's inspect some segmentations where we can observe misalignment
shapeIdxs = [8,9,10]

shapeSegSubset   = [shapeSegList[shapeIdx] for shapeIdx in shapeIdxs ]
shapeNamesSubset = [shapeNames[shapeIdx]   for shapeIdx in shapeIdxs ]

To inspect how mutliple segmentation are spatially aligned with respect to each other, we will visualize their surfaces in the same rendering window. 

In [ ]:
shapeSegIsosurfaces     = []
shapeSegIsosurfaces_vtk = []

for shapeSeg in shapeSegSubset:
    
    # extract isosurface
    shapeIsosurface = shapeSeg.toMesh(isovalue = isoValue)
    
    shapeSegIsosurfaces.append(shapeIsosurface)
 
    # sw to vtk
    shapeSegIsosurfaces_vtk.append(sw2vtkMesh(shapeIsosurface, verbose = False))


In [ ]:
# now visualize them on the same rendering window using itkwidgets
# note that they are not roughly alignemd  

# visualize with axes and auto rotation
itkw.view(  geometries     = shapeSegIsosurfaces_vtk, 
            rotate         = True, # enable auto rotation
            axes           = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-iso-align.mp4" autoplay muted loop controls style="width:100%"></p>

In [ ]:
# let's use pyvista and assign different colors for each mesh
# note this is very similar to the plot_volumes but with meshes rather than volumes

# helper function to add and plot a list of meshes
def plot_meshes(meshList,           # list of shapeworks meshes to be visualized
                meshNames       = None,  # list of strings of same size as shape list used to add text for each plot window, use None to not show text per window 
                use_same_window = False, # plot using multiple rendering windows if false
                is_interactive  = True,  # to enable interactive plots
                show_borders    = True,  # show borders for each rendering window
                meshes_color    = 'tan', # color to be used for meshes (can be a list with the same size as meshList if different colors are needed)
                mesh_style      = "surface", # visualization style of the mesh. style='surface', style='wireframe', style='points'. 
                show_mesh_edges = False, # show mesh edges
                opacities       = 1,     # opacity to be used for meshes (can be a list with the same size as meshList if different opacities are needed) 
                show_axes       = True,  # show a vtk axes widget for each rendering window
                show_bounds     = True,  # show volume bounding box
                show_all_edges  = True,  # add an unlabeled and unticked box at the boundaries of plot. 
                font_size       = 10,    # text font size for windows
                link_views      = True   # link all rendering windows so that they share same camera and axes boundaries
               ):
    
    num_samples = len(meshList)
    
    if meshNames is not None:
        if use_same_window and (len(meshNames) > 1):
            print('A single title needed when all meshes are to be displayed on the same window')
            return
        elif (not use_same_window) and  (len(meshNames) != num_samples):
            print('meshNames list is not consistent with number of samples')
            return
            
    if type(meshes_color) is not list: # single color given
        meshes_color = [meshes_color] * num_samples
        
    if type(opacities) is not list: # single opacity given
        opacities = [opacities] * num_samples
        
    if use_same_window:
        grid_rows, grid_cols = 1, 1
    else:
        # define grid size for the given number of samples
        grid_rows, grid_cols  = num_subplots(num_samples)

    # define the plotter
    plotter = pv.Plotter(shape    = (grid_rows, grid_cols),
                         notebook = is_interactive, 
                         border   = show_borders) 
    
    # add the given volume list (one at a time) to the plotter
    for meshIdx in range(num_samples):
        
        # which window to add the current mesh
        if use_same_window:
            rowIdx, colIdx = 0, 0
            titleIdx       = 0
        else:
            idUnraveled     = np.unravel_index(meshIdx, (grid_rows, grid_cols))
            rowIdx, colIdx  = idUnraveled[0], idUnraveled[1]
            titleIdx        = meshIdx
        
        # which title to use
        if meshNames is not None:
            meshName = meshNames[titleIdx]
        else:
            meshName = None

        # convert sw mesh to vtk mesh
        if type(meshList[meshIdx]) == sw.Mesh:
            mesh_vtk = sw2vtkMesh(meshList[meshIdx], 
                                  verbose = False)
        else:
            mesh_vtk = meshList[meshIdx]

        # add the current mesh
        add_mesh_to_plotter( plotter, mesh_vtk,   
                             rowIdx = rowIdx, colIdx = colIdx, 
                             title           = meshName,
                             mesh_color      = meshes_color[meshIdx],
                             mesh_style      = mesh_style,
                             show_mesh_edges = show_mesh_edges,
                             opacity         = opacities[meshIdx],
                             show_axes       = show_axes, 
                             show_bounds     = show_bounds, 
                             show_all_edges  = show_all_edges, 
                             font_size       = font_size)
        
    # link views
    if link_views and (not use_same_window):
        plotter.link_views()  

    # now, time to render our meshes
    plotter.show(use_ipyvtk = is_interactive)

In [ ]:
plot_meshes(shapeSegIsosurfaces,       
            use_same_window = True, 
            is_interactive  = True,  
            show_borders    = True,  
            meshes_color    = ['tan', 'blue','red'], 
            mesh_style      = "surface", 
            show_mesh_edges = False, 
            show_axes       = True,  
            show_bounds     = True,  
            show_all_edges  = True,  
            font_size       = 10,    
            link_views      = True   
           )

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-iso-align-pv.mp4" autoplay muted loop controls style="width:100%"></p>

### Too much background

Image boundaries are not tight around shapes, leaving irrelevant background voxels that might increase the memory footprint when optimizing the shape model. 

*We can crop segmentations to remove unnecessary background.*

In [ ]:
shapeIdx = 12
shapeSeg = shapeSegList[shapeIdx]
 
shapeSeg_vtk = sw2vtkImage(shapeSeg)

# to visualize label map - use label map blend 
itkw.view( image          = shapeSeg_vtk, # for orthoginal image plane
           label_image    = shapeSeg_vtk,  # for volume rendering segmentation
           slicing_planes = True, 
           axes           = True,
           rotate         = True, # enable auto rotation
           interpolation  = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-explore-bg.mp4" autoplay muted loop controls style="width:100%"></p>

### Binary segmentations

In general, this binary representation is not useful for finite numerical calculation of surface geometry and features that are required in shape modeling, which assumes the image is a sampling of a smooth function. 

Hence, ShapeWorks makes use of the signed distance transform of the binary segmentation that does satisfy this criterion. 

*For the correspondence optimization step, shapes can be represented as the zero level set of a smooth signed distance transform.*

### Tentative grooming

Hence, a tentative grooming pipeline entails the following steps:   
1. Resampling segmentations to have smaller and isotropic voxel spacing   
2. Rigidly aligning shapes   
3. Cropping and padding segmentations   
4. Converting segmentations to smooth signed distance transforms   


Let the fun begins!!! Please visit [Getting Started with Grooming Segmentations](getting-started-with-grooming-segmentations.ipynb) to learn how to groom your dataset.